In [37]:
import pandas as pd
from pandas_profiling import ProfileReport

import os
cwd = os.getcwd()
print(cwd)

C:\Users\alinu\Documents\__19fall\research\awid\project\src\new


In [38]:
f = open('../../data/features.csv', 'r')
feat = f.read().splitlines() 
f.close()

In [ ]:
feat

In [39]:
# import a training set
awid = pd.read_csv("../../data/AWID-CLS-R-Trn.csv", header=None, names=feat,low_memory=False)

# see the number of rows/columns
awid.shape

(1795575, 155)

In [40]:
# they use ? as a null attribute. We will eventually have to replace them with None values
awid.head()

,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,...,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated1,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated2,data.len,class
0,0,?,0.0,1.393661e+09,0.000000,0.000000,0.000000,261,261,0,...,?,?,?,?,?,?,?,?,?,normal
1,0,?,0.0,1.393661e+09,0.024271,0.024271,0.024271,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
2,0,?,0.0,1.393661e+09,0.001631,0.001631,0.025902,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
3,0,?,0.0,1.393661e+09,0.055325,0.055325,0.081227,159,159,0,...,?,?,?,?,?,?,?,?,?,normal
4,0,?,0.0,1.393661e+09,0.000415,0.000415,0.081642,54,54,0,...,?,?,?,?,?,?,?,?,?,normal


In [41]:
# see the distribution of response vars
awid['class'].value_counts(normalize=True)

normal           0.909564
injection        0.036411
impersonation    0.027023
flooding         0.027002
Name: class, dtype: float64

In [ ]:
%%time
profile = ProfileReport(awid, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile=

In [42]:
# they use ? as a null attribute. We will eventually have to replace them with None values
awid.head()

,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,...,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated1,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated2,data.len,class
0,0,?,0.0,1.393661e+09,0.000000,0.000000,0.000000,261,261,0,...,?,?,?,?,?,?,?,?,?,normal
1,0,?,0.0,1.393661e+09,0.024271,0.024271,0.024271,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
2,0,?,0.0,1.393661e+09,0.001631,0.001631,0.025902,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
3,0,?,0.0,1.393661e+09,0.055325,0.055325,0.081227,159,159,0,...,?,?,?,?,?,?,?,?,?,normal
4,0,?,0.0,1.393661e+09,0.000415,0.000415,0.081642,54,54,0,...,?,?,?,?,?,?,?,?,?,normal


In [ ]:
%%time
profile = ProfileReport(awid, minimal=True)
profile.to_file(output_file="output.html")
profile

In [43]:
#awid=awid.drop(['wlan_mgt.ssid'], axis=1)

sel_columns=['radiotap.present.reserved','wlan.fc.type_subtype','wlan.fc.ds','wlan_mgt.fixed.capabilities.cfpoll.ap','wlan_mgt.fixed.timestamp',\
            'wlan_mgt.tim.bmapctl.offset','wlan_mgt.rsn.capabilities.ptksa_replay_counter','wlan_mgt.rsn.capabilities.gtksa_replay_counter',\
            'wlan.wep.iv','wlan.wep.icv', 'wlan.qos.ack','wlan.qos.ack','wlan.bar.type','wlan.bar.compressed.tidinfo','wlan_mgt.fixed.listen_ival',\
            'wlan_mgt.fixed.status_code','wlan_mgt.fixed.aid','wlan_mgt.fixed.reason_code','wlan_mgt.fixed.auth_seq','wlan_mgt.fixed.htact',\
            'wlan_mgt.fixed.chanwidth','wlan_mgt.country_info.environment','wlan.tkip.extiv','wlan.ccmp.extiv']
remove_col_columns=['wlan.sa','wlan.da','wlan.ra','wlan.ta','wlan.bssid','wlan.ba.bm','wlan_mgt.fixed.current_ap']

awid = awid.apply(lambda x: x.replace("?","0x00") if x.name in sel_columns else x)
awid = awid.apply(lambda x: x.astype(str) if x.name in sel_columns else x)
awid = awid.apply(lambda x: x.apply(int, base=16) if x.name in sel_columns else x)

awid = awid.apply(lambda x: x.map(lambda x: x.replace(':','')) if x.name in remove_col_columns else x)
awid = awid.apply(lambda x: x.replace("?","0x00") if x.name in remove_col_columns else x)
awid = awid.apply(lambda x: x.astype(str) if x.name in remove_col_columns else x)
awid = awid.apply(lambda x: x.apply(int, base=16) if x.name in remove_col_columns else x)

awid=awid.drop(['wlan_mgt.ssid'], axis=1)

In [44]:
# replace the ? marks with 0 not with None
awid.replace({"?": 0}, inplace=True)

# transform all columns into numerical dtypes
for col in awid.columns:
    awid[col] = pd.to_numeric(awid[col], errors='ignore')

In [45]:
awid.head()

,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,...,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated1,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated2,data.len,class
0,0,0,0.0,1.393661e+09,0.000000,0.000000,0.000000,261,261,0,...,0,0,0,0,0,0,0,0,0,normal
1,0,0,0.0,1.393661e+09,0.024271,0.024271,0.024271,185,185,0,...,0,0,0,0,0,0,0,0,0,normal
2,0,0,0.0,1.393661e+09,0.001631,0.001631,0.025902,185,185,0,...,0,0,0,0,0,0,0,0,0,normal
3,0,0,0.0,1.393661e+09,0.055325,0.055325,0.081227,159,159,0,...,0,0,0,0,0,0,0,0,0,normal
4,0,0,0.0,1.393661e+09,0.000415,0.000415,0.081642,54,54,0,...,0,0,0,0,0,0,0,0,0,normal


In [46]:
awid.select_dtypes(include='object')

,class
0,normal
1,normal
2,normal
3,normal
4,normal
...,...
1795570,normal
1795571,normal
1795572,normal
1795573,normal


In [ ]:
awid.loc[:, (awid != 0).any(axis=0)]

In [ ]:
awid.shape

In [ ]:
%%time
profile = ProfileReport(awid, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile

In [47]:
# claims there are no null values because of the ?'s'
awid.isna().sum()

frame.interface_id               0
frame.dlt                        0
frame.offset_shift               0
frame.time_epoch                 0
frame.time_delta                 0
                                ..
wlan.qos.bit4                    0
wlan.qos.txop_dur_req            0
wlan.qos.buf_state_indicated2    0
data.len                         0
class                            0
Length: 154, dtype: int64

In [ ]:
# replace the ? marks with 0 not with None
awid.replace({"?": 0}, inplace=True)

In [ ]:
# Many missing pieces of data!
awid.isna().sum()

In [ ]:
# goal is to remove columns that have over 50% of missing data
columns_with_mostly_null_data = awid.columns[awid.isnull().mean() >= 0.5]

# 72 columns are going to be affected!
columns_with_mostly_null_data.shape

In [ ]:
# drop the columns with over half missing data
awid.drop(columns_with_mostly_null_data, axis=1, inplace=True)

In [ ]:
awid.shape

In [ ]:
# now drop the rows that have missing values
awid.dropna(inplace=True)  # drop rows with null data

In [48]:
# lost 456,169 rows
awid.shape

(1795575, 154)

In [ ]:
# doesn't affect our distribution too much

# 0.878763  is our null accuracy. Our model must be better than this number to be a contender
awid['class'].value_counts(normalize=True)

In [ ]:
# only select numerical columns for our ML algorithms, there should be more..
awid.select_dtypes(['number']).shape

In [ ]:
# transform all columns into numerical dtypes
for col in awid.columns:
    awid[col] = pd.to_numeric(awid[col], errors='ignore')

In [ ]:
# that makes more sense
awid.select_dtypes(['number']).shape

In [ ]:
# basic descroptive statistics
awid.describe()

In [52]:
X, y = awid.select_dtypes(['number']), awid['class']

In [53]:
import pickle
with open('../../data/pickle/train.pk', 'wb') as f:
    pickle.dump([X,y], f)

In [54]:
# read in the test data and do the same transformations to it to match the training data
awid_test = pd.read_csv('../../data/AWID-CLS-R-Tst.csv', header=None, names=feat,low_memory=False)

In [55]:
#awid=awid.drop(['wlan_mgt.ssid'], axis=1)

sel_columns=['radiotap.present.reserved','wlan.fc.type_subtype','wlan.fc.ds','wlan_mgt.fixed.capabilities.cfpoll.ap','wlan_mgt.fixed.timestamp',\
            'wlan_mgt.tim.bmapctl.offset','wlan_mgt.rsn.capabilities.ptksa_replay_counter','wlan_mgt.rsn.capabilities.gtksa_replay_counter',\
            'wlan.wep.iv','wlan.wep.icv', 'wlan.qos.ack','wlan.qos.ack','wlan.bar.type','wlan.bar.compressed.tidinfo','wlan_mgt.fixed.listen_ival',\
            'wlan_mgt.fixed.status_code','wlan_mgt.fixed.aid','wlan_mgt.fixed.reason_code','wlan_mgt.fixed.auth_seq','wlan_mgt.fixed.htact',\
            'wlan_mgt.fixed.chanwidth','wlan_mgt.country_info.environment','wlan.tkip.extiv','wlan.ccmp.extiv']
remove_col_columns=['wlan.sa','wlan.da','wlan.ra','wlan.ta','wlan.bssid','wlan.ba.bm','wlan_mgt.fixed.current_ap']

awid_test = awid_test.apply(lambda x: x.replace("?","0x00") if x.name in sel_columns else x)
awid_test = awid_test.apply(lambda x: x.astype(str) if x.name in sel_columns else x)
awid_test = awid_test.apply(lambda x: x.apply(int, base=16) if x.name in sel_columns else x)

awid_test = awid_test.apply(lambda x: x.map(lambda x: x.replace(':','')) if x.name in remove_col_columns else x)
awid_test = awid_test.apply(lambda x: x.replace("?","0x00") if x.name in remove_col_columns else x)
awid_test = awid_test.apply(lambda x: x.astype(str) if x.name in remove_col_columns else x)
awid_test = awid_test.apply(lambda x: x.apply(int, base=16) if x.name in remove_col_columns else x)

awid_test = awid_test.drop(['wlan_mgt.ssid'], axis=1)
# replace the ? marks with 0 not with None
awid_test.replace({"?": 0}, inplace=True)

# transform all columns into numerical dtypes
for col in awid_test.columns:
    awid_test[col] = pd.to_numeric(awid_test[col], errors='ignore')

In [ ]:
awid_test['radiotap.present.reserved']= awid_test['radiotap.present.reserved'].astype(str)
awid_test['radiotap.present.reserved']= awid_test['radiotap.present.reserved'].apply(int, base=16)

awid_test['wlan.fc.type_subtype']= awid_test['wlan.fc.type_subtype'].astype(str)
awid_test['wlan.fc.type_subtype']= awid_test['wlan.fc.type_subtype'].apply(int, base=16)

awid_test['wlan.fc.ds']= awid_test['wlan.fc.ds'].astype(str)
awid_test['wlan.fc.ds']= awid_test['wlan.fc.ds'].apply(int, base=16)

awid_test['wlan_mgt.fixed.capabilities.cfpoll.ap'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan_mgt.fixed.capabilities.cfpoll.ap']= awid_test['wlan_mgt.fixed.capabilities.cfpoll.ap'].fillna(0).astype(str)
#awid['wlan_mgt.fixed.capabilities.cfpoll.ap'].replace({"?": (lambda x: int(0, 16))}, inplace=True)
awid_test['wlan_mgt.fixed.capabilities.cfpoll.ap']= awid_test['wlan_mgt.fixed.capabilities.cfpoll.ap'].apply(int, base=16)

awid_test['wlan_mgt.fixed.timestamp'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan_mgt.fixed.timestamp']= awid_test['wlan_mgt.fixed.timestamp'].fillna(0).astype(str)
awid_test['wlan_mgt.fixed.timestamp']= awid_test['wlan_mgt.fixed.timestamp'].apply(int, base=16)

awid_test['wlan_mgt.tim.bmapctl.offset'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan_mgt.tim.bmapctl.offset']= awid_test['wlan_mgt.tim.bmapctl.offset'].fillna(0).astype(str)
awid_test['wlan_mgt.tim.bmapctl.offset']= awid_test['wlan_mgt.tim.bmapctl.offset'].apply(int, base=16)

awid_test['wlan_mgt.rsn.capabilities.ptksa_replay_counter'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan_mgt.rsn.capabilities.ptksa_replay_counter']= awid_test['wlan_mgt.rsn.capabilities.ptksa_replay_counter'].fillna(0).astype(str)
awid_test['wlan_mgt.rsn.capabilities.ptksa_replay_counter']= awid_test['wlan_mgt.rsn.capabilities.ptksa_replay_counter'].apply(int, base=16)

awid_test['wlan_mgt.rsn.capabilities.gtksa_replay_counter'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan_mgt.rsn.capabilities.gtksa_replay_counter']= awid_test['wlan_mgt.rsn.capabilities.gtksa_replay_counter'].fillna(0).astype(str)
awid_test['wlan_mgt.rsn.capabilities.gtksa_replay_counter']= awid_test['wlan_mgt.rsn.capabilities.gtksa_replay_counter'].apply(int, base=16)

awid_test['wlan.wep.iv'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan.wep.iv']= awid_test['wlan.wep.iv'].fillna(0).astype(str)
awid_test['wlan.wep.iv']= awid_test['wlan.wep.iv'].apply(int, base=16)

awid_test['wlan.wep.icv'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan.wep.icv']= awid_test['wlan.wep.icv'].fillna(0).astype(str)
awid_test['wlan.wep.icv']= awid_test['wlan.wep.icv'].apply(int, base=16)

awid_test['wlan.qos.ack'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan.qos.ack']= awid_test['wlan.qos.ack'].fillna(0).astype(str)
awid_test['wlan.qos.ack']= awid_test['wlan.qos.ack'].apply(int, base=16)

awid_test['wlan.sa']=awid_test['wlan.sa'].map(lambda x: x.replace(':',''))
awid_test['wlan.sa'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan.sa']= awid_test['wlan.sa'].fillna(0).astype(str)
awid_test['wlan.sa']= awid_test['wlan.sa'].apply(int, base=16)

awid_test['wlan.da']=awid_test['wlan.da'].map(lambda x: x.replace(':',''))
awid_test['wlan.da'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan.da']= awid_test['wlan.da'].fillna(0).astype(str)
awid_test['wlan.da']= awid_test['wlan.da'].apply(int, base=16)

awid_test['wlan.bssid']=awid_test['wlan.bssid'].map(lambda x: x.replace(':',''))
awid_test['wlan.bssid'].replace({"?": "0x00"}, inplace=True)
awid_test['wlan.bssid']= awid_test['wlan.bssid'].fillna(0).astype(str)
awid_test['wlan.bssid']= awid_test['wlan.bssid'].apply(int, base=16)

In [ ]:
# drop the problematic columns
awid_test.drop(columns_with_mostly_null_data, axis=1, inplace=True)
# replace ? with None
awid_test.replace({"?": None}, inplace=True)
# drop the rows with null data
awid_test.dropna(inplace=True)  # drop rows with null data
# convert columns to numerical values
for col in awid_test.columns:
    awid_test[col] = pd.to_numeric(awid_test[col], errors='ignore')
awid_test.shape

In [56]:
X_test = awid_test.select_dtypes(['number'])
y_test = awid_test['class']

In [58]:
with open('../../data/pickle/test.pk', 'wb') as f:
    pickle.dump([X_test,y_test], f)

In [59]:
# do a basic naive bayes fitting
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

# fit our model to the data
nb.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [60]:
# basic metric, accuracy
from sklearn.metrics import accuracy_score

# simple function to test the accuracy of a model fitted on training data on our testing data
def get_test_accuracy_of(model):
    y_preds = model.predict(X_test)
    return accuracy_score(y_preds, y_test)
    
# naive abyes does very poorly on its own!
get_test_accuracy_of(nb)

0.1747923626275313

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded_y = encoder.fit_transform(y)
encoded_y.shape

In [ ]:
encoded_y

In [ ]:
from sklearn.preprocessing import LabelBinarizer
binarizer = LabelBinarizer()
binarized_y = binarizer.fit_transform(encoded_y)
binarized_y.shape

In [ ]:
binarized_y[:5,]

In [ ]:
y.head()

In [ ]:
print (encoder.classes_)
print (binarizer.classes_)